In [14]:
# Polynomial, and thus integer, long division

import math
import numpy as np

# polynomials here are stored constant on the left

# O(1)
def poly_length(p):
    return len(p)

# O(n)
def poly_scale(p, n):
    if n >= 0:
        return [0] * n + p 
    
    # because n is negative, it discards the first n digits
    return p[-n:]

# O(n)
def poly_scalar(p, s):
    return [k * s for k in p]

# O(n)
def poly_pad(p, d):
    return p + [0] * (d - len(p))

# O(n)
def poly_norm(p):
    for i in reversed(range(len(p))):
        if p[i] != 0:
            return p[:i+1]
        
    return []

# O(n)
def poly_add(u, v):
    d = max(len(u), len(v))

    return poly_norm([a + b for a,b in zip(poly_pad(u, d), poly_pad(v, d))])

# O(n)
def poly_sub(u, v):
    d = max(len(u), len(v))

    return poly_norm([a - b for a,b in zip(poly_pad(u, d), poly_pad(v, d))])

# O(n log n)
def poly_mul(u, v):
    if len(u) == 0 or len(v) == 0:
        return []
    
    d = len(u) + len(v) - 1
    U = np.fft.fft(poly_pad(u, d))
    V = np.fft.fft(poly_pad(v, d))
    result = list(np.fft.ifft(U * V).real)

    return [x for x in result]


# O(n log n)
def poly_recip(p):
    # compute the reciprocal of polynomial P(x) of length n, a power of 2
    # the result is x^(2n-2) / p(x), that is, the recip shifted left
    # by 2n-2
    # fucking magic

    # divide leading
    scale = 1
    if p[-1] != 1:
        scale = 1/p[-1]
        p = poly_scalar(p, scale)


    n = len(p)

    # base case
    if n == 1:
        return [1/p[0]]
    
    # O(n log n)
    # compute the recip of the higher n/2 terms
    q = poly_recip(p[n//2:])
    # O(n log n)
    r = poly_sub(
        poly_scale(poly_scalar(q, 2), 3*n//2 - 2),  #2q * x^(3/2 n - 2)
        poly_mul(
            poly_mul(q, q),
            p
        )
        # qq p
    )

    # so the complexity add to O(n log n)

    # divide by x^(-(n-2))
    result = poly_scale(r, -n+2)

    result = poly_scalar(result, scale)

    return result


# (1+2x) * (1+3x)
# poly_mul(p1, p2)
# poly_recip([1, 1])


def poly_div(u, v):
    if len(v) == 0:
        raise ZeroDivisionError()
    
    if len(u) == 0:
        return []
    
    scale = 1
    if v[-1] != 1:
        scale = 1/v[-1]
        v = poly_scalar(v, scale)

    # scale both polynomial to powers of 2    
    d = 2**math.ceil(math.log(len(v), 2)) - len(v)
    scaled_u = poly_scale(u, d)
    scaled_v = poly_scale(v, d)
    padded_u = len(u) + d - 1
    padded_v = len(v) + d - 1

    s = poly_recip(scaled_v)
    # print(len(scaled_v))
    # print(poly_mul(scaled_u, s))
    # print(padded_v)
    q = poly_scale(poly_mul(scaled_u, s), -2*padded_v)
    # print(s)
    # print(poly_mul(scaled_u, s))
    # compute q and unpadded it from the padded inverse
    # when len(u) > 2 * len(v)
    # ?
    if padded_u > 2 * padded_v:
        t = poly_sub(poly_scale([1], 2*padded_v), poly_mul(s, scaled_v))
        q2 = poly_div(
            poly_scale(
                poly_mul(scaled_u, t),
                -2*padded_u
            ),
            scaled_v
        )
        q = poly_add(q, q2)

    q = poly_scalar(q, scale)
    return q


def combine(result):
    total = 0

    i = 0
    for digit in result:
        total += digit * 10**i
        i += 1

    return total


# p1 = [1, 2]
# p2 = [1, 3]
combine(poly_div([7, 2, 5, 1,1], [2,3,1]))
# poly_recip(p1)
# poly_scale([1, 2, 3], -2)




88.99999999999989